In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [25]:
# Cargar datos de divorcios

df_div = pd.read_csv('div_full.csv')
print(f"Shape divorcios: {df_div.shape}")
print(f"Columnas: {df_div.columns.tolist()}")
print()

# Cargar datos de matrimonios

df_mat = pd.read_csv('mat_full.csv')
print(f"Shape matrimonios: {df_mat.shape}")
print(f"Columnas: {df_mat.columns.tolist()}")

Shape divorcios: (56349, 12)
Columnas: ['AÑOREG', 'DEPOCU', 'DEPREG', 'DIAOCU', 'EDADHOM', 'EDADMUJ', 'MESOCU', 'MESREG', 'MUPOCU', 'MUPREG', 'NACHOM', 'NACMUJ']

Shape matrimonios: (752264, 15)
Columnas: ['AÑOREG', 'CLAUNI', 'DEPOCU', 'DEPREG', 'DIAOCU', 'EDADHOM', 'EDADMUJ', 'ESCHOM', 'ESCMUJ', 'MESOCU', 'MESREG', 'MUPOCU', 'MUPREG', 'NACHOM', 'NACMUJ']


In [26]:
# Convertir columnas numéricas (edades, años, días)

# Columnas numéricas en ambos datasets
columnas_numericas_div = ['AÑOREG', 'DIAOCU', 'EDADHOM', 'EDADMUJ']
columnas_numericas_mat = ['AÑOREG', 'DIAOCU', 'EDADHOM', 'EDADMUJ']

print("Convirtiendo columnas numéricas - DIVORCIOS")
for col in columnas_numericas_div:
    if col in df_div.columns:
        df_div[col] = pd.to_numeric(df_div[col], errors='coerce')
        validos = df_div[col].notna().sum()
        print(f"  {col}: {validos}/{len(df_div)} válidos ({validos/len(df_div)*100:.1f}%)")

print("\nConvirtiendo columnas numéricas - MATRIMONIOS")
for col in columnas_numericas_mat:
    if col in df_mat.columns:
        df_mat[col] = pd.to_numeric(df_mat[col], errors='coerce')
        validos = df_mat[col].notna().sum()
        print(f"  {col}: {validos}/{len(df_mat)} válidos ({validos/len(df_mat)*100:.1f}%)")


Convirtiendo columnas numéricas - DIVORCIOS
  AÑOREG: 56349/56349 válidos (100.0%)
  DIAOCU: 56349/56349 válidos (100.0%)
  EDADHOM: 25467/56349 válidos (45.2%)
  EDADMUJ: 25584/56349 válidos (45.4%)

Convirtiendo columnas numéricas - MATRIMONIOS
  AÑOREG: 752264/752264 válidos (100.0%)
  DIAOCU: 752264/752264 válidos (100.0%)
  EDADHOM: 751969/752264 válidos (100.0%)
  EDADMUJ: 751931/752264 válidos (100.0%)


In [27]:
#Limpieza de nombres de departamentos

mapeo_departamentos = {
    'Guatemala': 'Guatemala',
    'Quetzaltenango': 'Quetzaltenango',
    'Escuintla': 'Escuintla',
    'Jutiapa': 'Jutiapa',
    'San Marcos': 'San Marcos',
    'Izabal': 'Izabal',
    'Huehuetenango': 'Huehuetenango',
    'Suchitepequez': 'Suchitepéquez',
    'Suchitepéquez': 'Suchitepéquez',
    'Retalhuleu': 'Retalhuleu',
    'Zacapa': 'Zacapa',
    'Santa Rosa': 'Santa Rosa',
    'Chiquimula': 'Chiquimula',
    'Alta Verapaz': 'Alta Verapaz',
    'Jalapa': 'Jalapa',
    'Peten': 'Petén',
    'Petén': 'Petén',
    'Quiche': 'Quiché',
    'Quiché': 'Quiché',
    'Chimaltenango': 'Chimaltenango',
    'Sacatepequez': 'Sacatepéquez',
    'Sacatepéquez': 'Sacatepéquez',
    'Baja Verapaz': 'Baja Verapaz',
    'El Progreso': 'El Progreso',
    'Totonicapan': 'Totonicapán',
    'Totonicapán': 'Totonicapán',
    'Solola': 'Sololá',
    'Sololá': 'Sololá'
}

In [28]:
# Aplicar a divorcios
print("LIMPIEZA DE DEPARTAMENTOS - DIVORCIOS")
print("Valores únicos antes:")
print(df_div['DEPREG'].unique())

df_div['DEPREG_LIMPIO'] = df_div['DEPREG'].map(mapeo_departamentos).fillna(df_div['DEPREG'])

print("\nValores únicos después:")
print(df_div['DEPREG_LIMPIO'].unique())

# Identificar valores no oficiales (como "Ignorado" o "No especificado")
oficiales = set(mapeo_departamentos.values())
no_oficiales_div = set(df_div['DEPREG_LIMPIO'].unique()) - oficiales
print(f"\nValores no oficiales en divorcios: {no_oficiales_div}")

LIMPIEZA DE DEPARTAMENTOS - DIVORCIOS
Valores únicos antes:
<StringArray>
['Quetzaltenango',      'Guatemala',         'Izabal',         'Jalapa',
   'Sacatepequez',        'Jutiapa',     'Santa Rosa',   'Baja Verapaz',
  'Chimaltenango',  'Suchitepequez',  'Huehuetenango',         'Zacapa',
     'San Marcos',   'Alta Verapaz',     'Retalhuleu',      'Escuintla',
         'Quiche',    'El Progreso',     'Chiquimula',          'Peten',
    'Totonicapan',         'Solola',         'Sololá',    'Totonicapán',
         'Quiché',          'Petén',   'Sacatepéquez',  'Suchitepéquez']
Length: 28, dtype: str

Valores únicos después:
<StringArray>
['Quetzaltenango',      'Guatemala',         'Izabal',         'Jalapa',
   'Sacatepéquez',        'Jutiapa',     'Santa Rosa',   'Baja Verapaz',
  'Chimaltenango',  'Suchitepéquez',  'Huehuetenango',         'Zacapa',
     'San Marcos',   'Alta Verapaz',     'Retalhuleu',      'Escuintla',
         'Quiché',    'El Progreso',     'Chiquimula',       

In [29]:
# Aplicar a matrimonios
print("\n" + "="*60)
print("LIMPIEZA DE DEPARTAMENTOS - MATRIMONIOS")
print("Valores únicos antes:")
print(df_mat['DEPREG'].unique())

df_mat['DEPREG_LIMPIO'] = df_mat['DEPREG'].map(mapeo_departamentos).fillna(df_mat['DEPREG'])

print("\nValores únicos después:")
print(df_mat['DEPREG_LIMPIO'].unique())

no_oficiales_mat = set(df_mat['DEPREG_LIMPIO'].unique()) - oficiales
print(f"\nValores no oficiales en matrimonios: {no_oficiales_mat}")


LIMPIEZA DE DEPARTAMENTOS - MATRIMONIOS
Valores únicos antes:
<StringArray>
[     'Guatemala',    'Totonicapan',     'Santa Rosa',      'Escuintla',
  'Suchitepequez',   'Sacatepequez',         'Izabal',     'Retalhuleu',
        'Jutiapa', 'Quetzaltenango',   'Alta Verapaz',  'Huehuetenango',
     'San Marcos',     'Chiquimula',    'El Progreso',          'Peten',
  'Chimaltenango',         'Jalapa',         'Zacapa',   'Baja Verapaz',
         'Solola',         'Quiche',   'Sacatepéquez',          'Petén',
  'Suchitepéquez',    'Totonicapán',         'Quiché',         'Sololá']
Length: 28, dtype: str

Valores únicos después:
<StringArray>
[     'Guatemala',    'Totonicapán',     'Santa Rosa',      'Escuintla',
  'Suchitepéquez',   'Sacatepéquez',         'Izabal',     'Retalhuleu',
        'Jutiapa', 'Quetzaltenango',   'Alta Verapaz',  'Huehuetenango',
     'San Marcos',     'Chiquimula',    'El Progreso',          'Petén',
  'Chimaltenango',         'Jalapa',         'Zacapa',   '

In [30]:
# Limpieza de meses (crear columna numérica)

# Mapeo de meses a número
meses_map = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
    'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}

In [31]:
# Divorcios
df_div['MES_NUM'] = df_div['MESOCU'].map(meses_map)
print("DIVORCIOS - Valores nulos en MES_NUM:", df_div['MES_NUM'].isna().sum())
# Eliminar filas con mes nulo (si son pocas)
df_div = df_div.dropna(subset=['MES_NUM'])
print(f"Shape divorcios después de limpiar meses: {df_div.shape}")

DIVORCIOS - Valores nulos en MES_NUM: 0
Shape divorcios después de limpiar meses: (56349, 14)


In [32]:
# Matrimonios
df_mat['MES_NUM'] = df_mat['MESOCU'].map(meses_map)
print("\nMATRIMONIOS - Valores nulos en MES_NUM:", df_mat['MES_NUM'].isna().sum())
df_mat = df_mat.dropna(subset=['MES_NUM'])
print(f"Shape matrimonios después de limpiar meses: {df_mat.shape}")


MATRIMONIOS - Valores nulos en MES_NUM: 0
Shape matrimonios después de limpiar meses: (752264, 17)


In [33]:
# Excluir valores no oficiales en departamentos

# Para el análisis geográfico, excluimos registros con departamentos no oficiales
# (como "Ignorado", "No especificado", etc.)

print("DIVORCIOS - Registros con departamento no oficial:")
if no_oficiales_div:
    mask_div = df_div['DEPREG_LIMPIO'].isin(no_oficiales_div)
    print(f"  {mask_div.sum()} registros ({mask_div.sum()/len(df_div)*100:.2f}%)")
    df_div = df_div[~mask_div]
    print(f"  Shape después de excluir: {df_div.shape}")
else:
    print("  No hay valores no oficiales en divorcios")

print("\nMATRIMONIOS - Registros con departamento no oficial:")
if no_oficiales_mat:
    mask_mat = df_mat['DEPREG_LIMPIO'].isin(no_oficiales_mat)
    print(f"  {mask_mat.sum()} registros ({mask_mat.sum()/len(df_mat)*100:.2f}%)")
    df_mat = df_mat[~mask_mat]
    print(f"  Shape después de excluir: {df_mat.shape}")
else:
    print("  No hay valores no oficiales en matrimonios")


DIVORCIOS - Registros con departamento no oficial:
  No hay valores no oficiales en divorcios

MATRIMONIOS - Registros con departamento no oficial:
  No hay valores no oficiales en matrimonios


# Clusters


## Estabilidad Matrimonial

### ¿Existen grupos de departamentos con patrones similares de estabilidad matrimonial, considerando la tasa de divorcios respecto a matrimonios y las edades al divorcio?
### Hipótesis: Los departamentos con mayor desarrollo urbano tienen tasas de divorcio más altas y edades de divorcio más elevadas.